<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/rb_UBS_summarisation_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic umap-learn hdbscan sentence-transformers
!pip install transformers torch
!pip install rouge_score
!pip install evaluate
!pip install --upgrade protobuf
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

In [2]:
!pip install tensorflow
import tensorflow as tf
import numpy as np
import random

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.0
    Uninstalling protobuf-6.30.0:
      Successfully uninstalled protobuf-6.30.0


In [3]:
import time
import torch
from google.colab import drive
import os
import sys
import pandas as pd
import re
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import evaluate
from rouge_score import rouge_scorer
from typing import List, Union, Optional
import logging

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
def reset_session():
    tf.keras.backend.clear_session()
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)

In [5]:
# creating a pdf reader object
df_qna = pd.read_csv('/content/sample_data/ubs_qa_df_preprocessed_ver2.csv', header=0)

print("Q&A DataFrame:")
display(df_qna.head(3))


Q&A DataFrame:


,filename,Quarter,Question,Question_cleaned,Analyst_Bank,Response,Response_cleaned,Executive
0,1q23-earnings-call-remarks.pdf,1Q23,"Chis Hallam, Goldman Sachs Yes. Good morning, ...",['okay thank you capital requirements know sit...,Goldman,"Okay. Thank you. On capital requirements, you ...",['chis hallam goldman sachs yes good morning e...,['Sergio P. Ermotti']
1,1q23-earnings-call-remarks.pdf,1Q23,Yeah. Thanks. Just two questions. The first on...,['so sarah take first question take secondso g...,JPMorgan,"So, Sarah, take the first question. I'll take ...",['yeah thanks two questions first one related ...,"['Sergio P. Ermotti', 'Sarah Youngwood']"
2,1q23-earnings-call-remarks.pdf,1Q23,"Yeah. Thank you. Good morning. Welcome back, S...",['thank you ryan good back interact well look ...,Bank of America,"Thank you, Ryan. It is good to be back to inte...",['yeah thank you good morning welcome back ser...,['Sergio P. Ermotti']


In [6]:
df_qna = df_qna[df_qna["Quarter"] != "1Q23"]

In [7]:
filtered_df = df_qna[ (df_qna["Quarter"] == "4Q24") & (df_qna["Analyst_Bank"] == "Bank of America")]

In [8]:
sentiment = filtered_df["Response"].tolist()  #### genertaing list for modeling

###**Summarisation model**

In [ ]:
reset_session()

In [9]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class TextSummarizer:
    def __init__(self, model_name: str = "google/flan-t5-large", device: Optional[str] = None):
        """Initialize the summarizer with model and device."""
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)
            logger.info(f"Successfully loaded {model_name}")
        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def chunk_text(self,
                  text: Union[str, List[str]],
                  chunk_size: int = 400,
                  overlap: int = 50) -> List[str]:
        """Split text into overlapping chunks."""
        # Validate parameters
        if chunk_size <= 0 or overlap < 0 or overlap >= chunk_size:
            raise ValueError("Invalid chunk_size or overlap parameters")

        try:

            if isinstance(text, list):
                text = " ".join(text)

            if not text.strip():
                return []

            words = text.split()
            chunks = []
            start = 0

            while start < len(words):
                end = min(start + chunk_size, len(words))
                chunk = " ".join(words[start:end])
                chunks.append(chunk)
                start += chunk_size - overlap

            logger.debug(f"Split text into {len(chunks)} chunks")
            return chunks

        except Exception as e:
            logger.error(f"Error in chunk_text: {str(e)}")
            raise

    def summarize_text(self,
                      text: str,
                      min_new_tokens: int = 100,
                      max_new_tokens: int = 400) -> str:
        """Summarize a single piece of text."""

        if pd.isna(text) or not text.strip():
            logger.warning("Empty or NaN text provided")
            return ""

        try:
            prompt = f"Rewrite the following text into a concise and original summary while maintaining its key ideas: {text}"


            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=512
            ).to(self.device)


            with torch.no_grad():
                outputs = self.model.generate(
                    inputs.input_ids,
                    min_new_tokens=min_new_tokens,
                    max_new_tokens=max_new_tokens,
                    num_beams=4,
                    length_penalty=2,
                    no_repeat_ngram_size=3,
                    early_stopping=True,
                    do_sample=False
                )

            summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return summary

        except Exception as e:
            logger.error(f"Error in summarize_text: {str(e)}")
            raise

    def summarize_long_text(self,
                          text: Union[str, List[str]],
                          chunk_size: int = 300,
                          overlap: int = 50) -> str:
        """Handle long text summarization."""
        try:
            # Get chunks
            chunks = self.chunk_text(text, chunk_size, overlap)
            if not chunks:
                logger.warning("No valid chunks to summarize")
                return ""

            # Summarize chunks
            chunk_summaries = []
            for i, chunk in enumerate(chunks):
                logger.debug(f"Summarizing chunk {i+1}/{len(chunks)}")
                summary = self.summarize_text(chunk)
                if summary.strip():
                    chunk_summaries.append(summary)

            if not chunk_summaries:
                logger.warning("No valid summaries generated")
                return ""

            # If single chunk, return its summary
            if len(chunk_summaries) == 1:
                return chunk_summaries[0]

            # Summarize the combined summaries
            logger.debug("Generating final summary")
            final_summary = self.summarize_text(
                " ".join(chunk_summaries),
                min_new_tokens=150,
                max_new_tokens=300
            )

            return final_summary

        except Exception as e:
            logger.error(f"Error in summarize_long_text: {str(e)}")
            raise

# Running the model
try:
    summarizer = TextSummarizer()

    # Perform summarization
    logger.info("Starting summarization of management discussion")
    summary_text = summarizer.summarize_long_text(sentiment)

    print("Final Summary:", summary_text)

    # Create DataFrame for summary
    df_summary = pd.DataFrame({"Summary": [summary_text]})
    df_summary.to_csv("summary_output.csv", index=False)  # Export to CSV

    # Save model and tokenizer
    model_save_path = "./saved_model"
    summarizer.model.save_pretrained(model_save_path)
    summarizer.tokenizer.save_pretrained(model_save_path)

    logger.info(f"Model and tokenizer saved to {model_save_path}")

except Exception as e:
    logger.error(f"Error during summarization: {str(e)}")
    print(f"An error occurred: {str(e)}")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Final Summary: UBS CEO: We're focusing on driving where possible, non-NII revenue growth improve on recurring revenue and P&C and also on transaction revenues. That will be a focus to partially offset the headwinds that we see in P & C. I mean as far as abilities to effectively hedge or extend duration, I mean that's just not when the rates are this low and with the flat yield curve, you know, extending duration, for example, on non-maturing deposits doesn't offer much of an advantage at all. To offset the capital repatriated to AG, its first tier subsidiary. To be clear, to offset the Capital Repatriation to the Group, its First Tier Subsidiary. The Group is a subsidiary of the Société Générale, which is the world's largest financial institution.


In [10]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [11]:
discussion_str = " ".join(sentiment)

# Calculate ROUGE scores
scores = scorer.score(discussion_str, summary_text)
for key in scores:
    print(f'{key}: {scores[key]}')

rouge1: Score(precision=0.8602941176470589, recall=0.4254545454545455, fmeasure=0.5693430656934307)
rouge2: Score(precision=0.7555555555555555, recall=0.3722627737226277, fmeasure=0.49877750611246935)
rougeL: Score(precision=0.7794117647058824, recall=0.38545454545454544, fmeasure=0.5158150851581509)


### **Running model on full period and crating a CSV output with data from each individual quarters on negative sentiment text.**

In [12]:
summarizer = TextSummarizer()

summary_data = []

quarter_years = df_qna["Quarter"].unique()
analysts = df_qna["Analyst_Bank"].unique()

# Loop through each unique Quarter-Year and Analyst
for quarter in quarter_years:
    for analyst in analysts:
        filtered_df = df_qna[(df_qna["Quarter"] == quarter) & (df_qna["Analyst_Bank"] == analyst)]

        snippets = filtered_df["Response"].dropna().astype(str).tolist()

        if not snippets:
            logger.warning(f"No valid snippets found for Quarter-Year: {quarter}, Analyst_Bank: {analyst}")
            continue

        print(type(summarizer))
        # Generate the summary for the snippets in this Quarter-Year and Analyst
        try:
            summary_text = summarizer.summarize_long_text(snippets)
        except Exception as e:
            logger.error(f"Error summarizing for Quarter {quarter}, Analyst_Bank {analyst}: {str(e)}")
            continue

        summary_data.append({"Summary": summary_text, "Quarter": quarter, "Analyst_Bank": analyst})

# Convert summary data to DataFrame
df_summary = pd.DataFrame(summary_data)

df_summary.to_csv("summary_output_UBS.csv", index=False)

print(df_summary)


<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>


...

<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>
<class '__main__.TextSummarizer'>


<class '__main__.TextSummarizer'>
                                              Summary Quarter  \
0   Alastair: Hey, Todd. Thanks for the questions....    1Q24   
1   opex progress that we saw, sort of informs a b...    1Q24   
2   I think that some of the conditions that were ...    1Q24   
3   Giulia: TRX in GWM was a very strong 1Q. In te...    1Q24   
4   I think that you know, at this stage, the only...    1Q24   
..                                                ...     ...   
74  On the US wealth side, I think it's just impor...    4Q24   
75  Global Wealth Management's expectations for th...    4Q24   
76  We've been quite focused continuing to do bala...    4Q24   
77  On the second question, we are targeting a low...    4Q24   
78  Kian: I can only reiterate that we don't reall...    4Q24   

           Analyst_Bank  
0       Bank of America  
1         Goldman Sachs  
2                    JP  
3        Morgan Stanley  
4     BNP Paribas Exane  
..                  ...  
74 